# Balance Sheet

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import chi2_contingency
from sklearn.cluster import KMeans
import matplotlib.dates as mdates
from statsmodels.stats.multicomp import pairwise_tukeyhsd

## Financial Accounting

In [16]:
financial_df  = pd.read_csv('../data/double_entry_financial_accounting.csv')
financial_df ['Date'] = pd.to_datetime(financial_df ['Date'])
financial_df ['Date'] = financial_df ['Date'] - pd.DateOffset(years=1)
financial_df .rename(columns={'Transaction_Type': 'Transaction Type', 'Payment_Method': 'Payment Method'}, inplace=True)

In [19]:
print(financial_df['Transaction Type'].unique())
print(financial_df['Category'].unique())
print(financial_df['Payment Method'].unique())
print(financial_df['Account'].unique())

['Transfer' 'Sale' 'Expense' 'Purchase']
['Asset' 'Expense' 'Liability' 'Revenue']
['Cash' 'Credit Card' 'Check' 'Bank Transfer']
['Accounts Receivable' 'Expense Account' 'Liability Account'
 'Asset Account' 'Sales Revenue' 'Accounts Payable' 'Inventory' 'Cash'
 'Revenue Account']


In [22]:
financial_df['Balance'] = financial_df['Debit'] - financial_df['Credit']
account_balances = financial_df.groupby('Account')['Balance'].sum()

assets = ['Accounts Receivable', 'Cash', 'Inventory', 'Asset Account']
liabilities = ['Accounts Payable', 'Liability Account']
equity = ['Sales Revenue', 'Expense Account', 'Revenue Account']

total_assets = account_balances[assets].sum()
total_liabilities = account_balances[liabilities].sum()
total_equity = account_balances[equity].sum()

if 'Equity' not in account_balances:
    total_equity = total_assets - total_liabilities

balance_sheet = pd.DataFrame({
    'Category': ['Assets', 'Liabilities', 'Equity'],
    'Total': [total_assets, total_liabilities, total_equity]
})

print(balance_sheet)

      Category       Total
0       Assets  6124640.37
1  Liabilities  1763241.09
2       Equity  4361399.28


In [20]:
"""
# Calculate the balances for each account
financial_df['Balance'] = financial_df['Debit'] - financial_df['Credit']
account_balances = financial_df.groupby(['Account', 'Category'])['Balance'].sum()

# Separate into Assets, Liabilities, and Equity
assets = account_balances[account_balances.index.get_level_values('Category') == 'Asset']
liabilities = account_balances[account_balances.index.get_level_values('Category') == 'Liability']
# Assuming Equity is calculated as a residual
equity = assets.sum() - liabilities.sum()

# Construct the Balance Sheet
balance_sheet = pd.DataFrame({
    'Assets': assets,
    'Liabilities': liabilities,
    'Equity': pd.Series({'Total Equity': equity})
})

# Fill NaNs with zeros for a cleaner presentation
balance_sheet.fillna(0, inplace=True)

# Display the Balance Sheet
print(balance_sheet)
"""

                                    Assets  Liabilities  Equity
Accounts Payable    Asset      -3877192.05         0.00     0.0
                    Liability         0.00         0.00     0.0
Accounts Receivable Asset      15272312.94         0.00     0.0
Asset Account       Asset         24507.99         0.00     0.0
Cash                Asset     -11252396.69         0.00     0.0
Inventory           Asset       3935363.86         0.00     0.0
Liability Account   Liability         0.00   3717127.19     0.0
T                   o                 0.00         0.00     0.0


c:\Users\misha\anaconda3\lib\site-packages\pandas\core\indexes\multi.py:3635: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  result = lib.fast_unique_multiple([self._values, rvals], sort=sort)


## Managerial Accounting

In [17]:
managerial_df  = pd.read_csv('../data/double_entry_managerial_accounting.csv')
managerial_df['Date'] = pd.to_datetime(managerial_df ['Date'])

In [18]:
print(managerial_df['Transaction Type'].unique())
print(managerial_df['Category'].unique())
print(managerial_df['Department'].unique())
print(managerial_df['Project'].unique())
print(managerial_df['Account'].unique())

['Expense' 'Liability Payment' 'Revenue' 'Asset Purchase']
['Utilities' 'Supplies' 'Salaries' 'Rent' 'Assets' 'Loans' 'Royalties'
 'Service Revenue' 'Product Sales']
['Marketing' 'HR' 'Sales' 'Finance']
['Project B' 'Project D' 'Project C' 'Project A']
['Expense Account' 'Cash' 'Asset Account' 'Revenue Account']


In [24]:
# Calculate the balances for each account
managerial_df['Balance'] = managerial_df['Debit'] - managerial_df['Credit']
account_balances = managerial_df.groupby('Account')['Balance'].sum()

# Define the categories for assets, liabilities, and equity
assets = ['Cash', 'Asset Account']
liabilities = ['Accounts Payable']
equity = ['Revenue Account', 'Expense Account']  # Modify as needed

# Calculate the total for each category
total_assets = account_balances.reindex(assets).sum()
total_liabilities = account_balances.reindex(liabilities).sum()
total_equity = account_balances.reindex(equity).sum()

# Equity might also be calculated as Assets minus Liabilities
if 'Equity Account' not in account_balances:
    total_equity = total_assets - total_liabilities

# Create the balance sheet
balance_sheet = pd.DataFrame({
    'Category': ['Assets', 'Liabilities', 'Equity'],
    'Total': [total_assets, total_liabilities, total_equity]
})

# Print the balance sheet
print(balance_sheet)

      Category        Total
0       Assets  57560296.75
1  Liabilities         0.00
2       Equity  57560296.75
